## 1046. Last Stone Weight
- Description:
  <blockquote>
    You are given an array of integers `stones` where `stones[i]` is the weight of the `i<sup>th</sup>` stone.

  We are playing a game with the stones. On each turn, we choose the **heaviest two stones** and smash them together. Suppose the heaviest two stones have weights `x` and `y` with `x <= y`. The result of this smash is:

  -   If `x == y`, both stones are destroyed, and
  -   If `x != y`, the stone of weight `x` is destroyed, and the stone of weight `y` has new weight `y - x`.

  At the end of the game, there is **at most one** stone left.

  Return _the weight of the last remaining stone_. If there are no stones left, return `0`.

  **Example 1:**

  ```
  Input: stones = [2,7,4,1,8,1]
  Output: 1
  Explanation: 
  We combine 7 and 8 to get 1 so the array converts to [2,4,1,1,1] then,
  we combine 2 and 4 to get 2 so the array converts to [2,1,1,1] then,
  we combine 2 and 1 to get 1 so the array converts to [1,1,1] then,
  we combine 1 and 1 to get 0 so the array converts to [1] then that's the value of the last stone.

  ```

  **Example 2:**

  ```
  Input: stones = [1]
  Output: 1

  ```

  **Constraints:**

  -   `1 <= stones.length <= 30`
  -   `1 <= stones[i] <= 1000`
  </blockquote>

- URL: [Problem_URL](https://leetcode.com/problems/last-stone-weight/description/)

- Topics: Max Heap, Counting / Bucket Sort

- Difficulty: Easy / Medium

- Resources: example_resource_URL

### Solution 1, Max Heap

Let N be the length of stones.

    Time complexity : O(NlogN).

    Converting an array into a Heap takes O(N) time (it isn't actually sorting; it's putting them into an order that allows us to get the maximums, each in O(logN) time).

    Like before, the main loop iterates up to N−1 times. This time however, it's doing up to three O(logN) operations each time; two removes, and an optional add. Like always, the three is an ignored constant. This means that we're doing N⋅O(logN)=O(NlogN) operations.

    Space complexity : O(N) or O(logN).

    In Python, heapq.heapify() is an in-place operation, so it uses O(1) auxiliary space. However, the heap still stores n elements within the input list, which contributes O(n) to total space complexity. So, the total space complexity is O(n).

In [ ]:
class Solution:
    def lastStoneWeight(self, stones: List[int]) -> int:
        if len(stones) == 1:
            return stones[0]

        heap = []

        for stone in stones:
            heapq.heappush(heap, -stone)

        while len(heap) > 1:
            top1 = heapq.heappop(heap)
            top2 = heapq.heappop(heap)

            if top1 != top2:
                heapq.heappush(heap, (top1-top2))
        
        return -heapq.heappop(heap) if len(heap) > 0  else 0



### Solution 2, Counting / bucket Sort

This bucket method is optimal when max weight is small (e.g., ≤1000). For arbitrary large weights, heap is better


    Why the if diff <= current_weight check?

    We’re scanning downward from high to low weights.
    If diff <= current_weight, then this new stone won’t be missed — we’ll encounter it later in the scan (since we haven’t passed diff yet).
    But if diff > current_weight, we’ve already passed that weight index! So we can’t put it in buckets (we’d skip it). Instead, we hold it as biggest_weight for future collisions.



Time complexity : O(N+W).

    Putting the N stones of the input array into the bucket array is O(N), because inserting each stone is an O(1) operation.

    In the worst case, the main loop iterates through all of the W indexes of the bucket array. Processing each bucket is an O(1) operation. This, therefore, is O(W).

    Seeing as we don't know which is larger out of N and W, we get a total of O(N+W).

    Technically, this algorithm is pseudo-polynomial, as its time complexity is dependent on the numeric value of the input. Pseudo-polynomial algorithms are useful when there is no "true" polynomial alternative, but in situations such as this one where we have an O(NlogN) alternative (Approach 3), they are only useful for very specific inputs.

    With the small values of W that your code is tested against for this question here on LeetCode, this approach turns out to be faster than Approach 3. But that does not make it the better approach.

    Space complexity : O(W).

    We allocated a new array of size W.

In [ ]:
class Solution:
    def lastStoneWeight(self, stones: List[int]) -> int:
        max_weight = max(stones)
        buckets = [0] * (max_weight + 1)
        
        # Count stones by weight
        for weight in stones:
            buckets[weight] += 1

        biggest = 0  # Holds the current unpaired heaviest stone
        weight = max_weight
        
        while weight > 0:
            if buckets[weight] == 0:
                weight -= 1
                continue

            if biggest == 0:
                # Cancel pairs of same-weight stones
                if buckets[weight] % 2 == 1:
                    biggest = weight
                weight -= 1
            else:
                # Collide 'biggest' with one stone of current weight
                buckets[weight] -= 1
                diff = biggest - weight
                
                if diff == 0:
                    biggest = 0  # Both stones destroyed
                elif diff <= weight:
                    # New stone can be processed later in scan
                    buckets[diff] += 1
                    biggest = 0
                else:
                    # New stone is heavier than current weight → keep as pending
                    biggest = diff
                # Important: do NOT decrement weight here!
                # There might be more stones at this weight to collide with new 'biggest'
                
        return biggest